Reading from recordings

In [44]:
from torch.utils.data import Dataset
import numpy as np
import processing
from torch.utils.data import DataLoader, random_split


class SensorDataset(Dataset):
    def __init__(self):
        self.samples = []

        recording_list = [
            {'title': 'recording_2022_05_25_11_55_19', 'start': 300, 'end': 400, 'label': 0},  # 0 people
            {'title': 'recording_2022_05_25_11_46_26', 'start': 300, 'end': 400, 'label': 1},  # 1 person
            {'title': 'recording_2022_05_25_11_47_20', 'start': 300, 'end': 400, 'label': 2},  # 2 people
            {'title': 'recording_2022_05_25_11_48_56', 'start': 300, 'end': 400, 'label': 3},  # 3 people
        ]

        for recording in recording_list:
            file_name = f'../../recordings/{recording["title"]}/RadarIfxAvian_00/radar.npy'
            data = np.load(file_name)
            start_frame, end_frame, label = recording["start"], recording["end"], recording["label"]

            # iterate frames
            for frame_ix, sensor_array in enumerate(data[start_frame:end_frame]):
                for sensor_ix, image in enumerate(sensor_array):
                    processed_image = processing.processing_rangeDopplerData(image)
                    self.samples.append((processed_image, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


dataset = SensorDataset()
print(len(dataset))
train_size, test_size = round(len(dataset) * 0.8), round(len(dataset) * 0.2)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

1200


In [47]:
input_size = 3 * 64 * 64
hidden_size = [128, 64]
output_size = 4
import torch.nn as nn
from torch import optim


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()

        # Inputs to hidden layer linear transformation
        self.layer1 = nn.Linear(input_size, hidden_size[0])
        # Hidden layer 1 to HL2 linear transformation
        self.layer2 = nn.Linear(hidden_size[0], hidden_size[1])
        # HL2 to output linear transformation
        self.layer3 = nn.Linear(hidden_size[1], output_size)

        # Define relu activation and LogSoftmax output
        self.relu = nn.ReLU()
        self.LogSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # HL1 with relu activation
        out = self.relu(self.layer1(x))
        # HL2 with relu activation
        out = self.relu(self.layer2(out))
        # Output layer with LogSoftmax activation
        out = self.LogSoftmax(self.layer3(out))
        return out


model = NeuralNet(input_size, hidden_size, output_size)
lossFunction = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

In [48]:
num_epochs = 10
for epoch in range(num_epochs):
    loss_ = 0
    for images, labels in train_loader:
        # Flatten the input images
        images = images.flatten()

        # Forward Pass
        output = model(images)
        # Loss at each iteration by comparing to target(label)
        loss = lossFunction(output, labels)

        # Backpropagation gradient of loss
        optimizer.zero_grad()
        loss.backward()

        # Updating parameters(weights and bias)
        optimizer.step()

        loss_ += loss.item()
    print("Epoch{}, Training loss:{}".format(epoch, loss_ / len(train_loader)))

RuntimeError: size mismatch, got 128, 128x12288,262144